In [1]:
import numpy as np
alldeploys=["Wizard","ManaTower","Executioner","Ghost"]
allabilities=["Fireball","Execute","Haunt","Nuke"]
allnotarget=["HealSpell","ExorcismSpell","ThornSpell"]
allmoves=['EndTurn']
alldeploysquares=[]
allhandcards=alldeploys+allnotarget
for card in alldeploys:
    alldeploysquares.extend([card+str(x) for x in range(4)])
allmoves.extend(alldeploysquares)
for move in allabilities:
    allmoves.extend([move+str(x) for x in range(4)])
allmoves.extend([allnotarget[x] for x in range(2)])
def encodehand(hand,length=len(allhandcards)):
    small=len(allhandcards)
    return [hand.count(x) if x in hand else 0 for x in allhandcards]+[0]*max(0,length-small)
def encodemove(move,length=len(allmoves)):
    small=len(allmoves)
    return [1 if x==move else 0 for x in allmoves]+[0]*max(0,length-small)
def encodemoves(moves,length=len(allmoves)):
    small=len(allmoves)
    return [1 if x in moves else 0 for x in allmoves]+[0]*max(0,length-small)
def encodedeployed(troops,length=len(alldeploysquares)):
    small=len(alldeploysquares)
    return [1 if x in troops else 0 for x in alldeploysquares]+[0]*max(0,length-small)
def encodehealths(troops):

    return [troop.health if hasattr(troop,"health") else 0 for troop in troops]+[0]*(4-len(troops))
snapshot = {

    "My_Mana": [0],
    "Their_Mana": [0],
    "My_Health": [0],
    "Their_Health": [0],
    "Their_Card_Types_Seen":encodehand([0]),
    "Their_Currently_Visible_Cards": encodedeployed([0,0,0,0]),
    "Their_cardhealths":[0]*4,
    "Your cardhealths":[0]*4,
    "Active_Cards":encodedeployed([0,0,0,0]),
    "Deck":encodehand([]),
    "Action Taken":encodemove([])
}

information=[]
for key in snapshot.keys():
    information.extend(snapshot[key])
infolength=len(information)
infolength

93

In [2]:
import time
# -----------------------------
# Game controller
# -----------------------------
class Game:
    def __init__(self, player_1, player_2, length=10, prnt=True, offset=[0,2],show=False):
        self.turn = 0
        self.prnt = prnt
        self.moves_played = 0
        self.length = length
        self.game_over = False
        self.players = [player_1, player_2]
        player_1.game = self
        player_2.game = self
        player_1.team = 0
        player_2.team = 1
        player_2.base_health += offset[0]
        player_2.mana += offset[1]
        self.boards=[]
        self.show=show
        self.player_1=player_1
        self.player_2=player_2
    def change_turn(self):
        self.turn = 1 - self.turn

    def play(self):
        start=time.time()
        while self.moves_played < self.length and not self.game_over:
            player = self.players[self.turn]
            opponent = self.players[self.turn-1]
            if self.prnt:
                print(f"\n--- Turn {self.moves_played}, Player {self.turn+1} ---")
            player.make_move(self)
            if self.show:
                self.show_board()
            if time.time()-start>20:
                return self.end_game()
            
        return self.end_game()

    def end_game(self):
        p1, p2 = self.players
        if p1.base_health > p2.base_health:
            return 1
        elif p1.base_health < p2.base_health:
            return -1
        return 0
    def show_board(self):
        print("----------------------------------------------")
        print("Current Board:")
        print(f"Player 2 Health:{self.player_2.base_health}")
        print(f"Player 2 Mana:{self.player_2.mana}")
        print("Player 2 cards in hand:")
        print(*[card.name for card in self.player_2.deck])
        print("Player 2 troops deployed:")
        print(*[square.troop.name if not square.empty else "____" for square in self.player_2.available_squares])
        print("Player 1 troops deployed:")
        print(*[square.troop.name if not square.empty else "____" for square in self.player_1.available_squares])
        print("Player 1 cards in hand:")
        print(*[card.name for card in self.player_1.deck])
        print(f"Player 1 Health:{self.player_1.base_health}")
        print(f"Player 1 Mana:{self.player_1.mana}")
        print("----------------------------------------------")

In [3]:
from collections import deque
# -----------------------------
# GamePlayer
# -----------------------------
class GamePlayer:
    def __init__(self, difficulty, game, deck, decision_maker,length=len(allmoves)):
        self.difficulty = difficulty
        self.game = game
        # Start with the full deck as "largedeck"
        self.largedeck = deque(deepcopy(deck))
        self.deck = []  # current hand (max 5)
        self.active_cards = []
        self.decision_maker = decision_maker
        self.team = None
        self.available_squares = [Square(i) for i in range(4)]
        self.mana = 30
        self.base_health = 15
        # draw initial hand
        self.draw_cards()
        self.move="EndTurn"
        snapshot = {
           
            "My_Mana": [self.mana],
            "Their_Mana": [self.mana],
            "My_Health": [self.base_health],
            "Their_Health": [self.base_health],
            "Their_Card_Types_Seen":encodehand([0]),
            "Their_Currently_Visible_Cards": encodedeployed([0,0,0,0]),
            "Their_cardhealths":[0]*4,
            "Your cardhealths":[0]*4,
            "Active_Cards":encodedeployed([0,0,0,0]),
            "Deck":encodehand(self.deck),
            "Action Taken":encodemove(self.move)
        }
        self.information=[]
        for key in snapshot.keys():
            self.information.extend(snapshot[key])
        self.known_enemy_types = set()
        self.passive_effectors=[]
        
    def clone(self):
        return deepcopy(self)

    def draw_cards(self):
        """Refill hand up to 5 cards from largedeck."""
        while len(self.deck) < 5 and self.largedeck:
            
            self.deck.append(self.largedeck.pop())

    def refresh_deck(self):
        """Call this at the start of a turn to ensure 5 cards in hand."""
        self.draw_cards()

    def get_possible_moves(self, game):
        moves = [EndTurnMove(self)]
        # Only consider cards that are alive
        extend_deck = [c for c in self.deck + self.active_cards]

        for card in extend_deck:
            moves.extend(card.get_possible_moves(game,self))

        return moves

    def make_move(self, game):
        moves = self.get_possible_moves(game)
        if moves:
            move = self.decision_maker.choose_move(moves,self.information)
            self.move=move.movename
            move.execute(game)
    

    def record_information(self, game,opponent,length=len(allmoves)):
        
        visible_enemy_cards = [c for c in opponent.active_cards if not getattr(c, "hidden", False)]

        # update memory of seen types
        self.known_enemy_types.update(c.name+str(c.square.index) for c in visible_enemy_cards)
        
        enemy_list=list(self.known_enemy_types) +[0]*(length-len(self.known_enemy_types))
        
        snapshot = {
           
            "My_Mana": [int(self.mana)],
            "Their_Mana": [int(opponent.mana)],
            "My_Health": [int(self.base_health)],
            "Their_Health": [int(opponent.base_health)],
            "Their_Card_Types_Seen":encodehand(enemy_list) ,
            "Their_Currently_Visible_Cards": encodedeployed([square.troop.name+str(square.index) if not square.empty else 0 for square in opponent.available_squares]),
            "Their_cardhealths":encodehealths(opponent.active_cards),
            "Your cardhealths":encodehealths(self.active_cards),
            "Active_Cards":encodedeployed([square.troop.name+str(square.index) if not square.empty else 0 for square in self.available_squares]),
            "Deck":encodehand([c.name for c in self.deck]),
            "Action Taken":encodemove(self.move)
            
        }
      
        self.information=[]
        for key in snapshot.keys():
            self.information.extend(snapshot[key])
    def invigorate(self):
        for card in self.active_cards:
            card.exhausted=False
    def employ_effects(self):
        for effect in self.passive_effectors:
            effect.tick(self)

In [4]:
class Effect:
    def __init__(self, owner, source=None):
        self.owner = owner      # usually the player
        self.source = source    # the troop or item that caused this effect
        self.active = True

    def tick(self, effects):
        pass


class ManaTowerEffect(Effect):
    def tick(self,player):
        if player.game.prnt:
            print(f"{self.source.name} grants +1 mana.")
        self.owner.mana += 1
class HiddenThornmailEffect(Effect):
    def tick(self,card,caster,damage):
        if card.player.game.prnt:
            print(f"Damage reduced from {damage} to {damage-2}")
        damage=max(0,damage-2)
        
            
        caster.take_damage(card,damage,card.player)
        if self in card.defeffects:
            card.defeffects.remove(self)
        self.active=False
        card.player.largedeck.append(ThornCard(card.team))
        card.damage=damage
        
        
    

In [5]:
from abc import ABC, abstractmethod
from copy import deepcopy

# -----------------------------
# Card base class
# -----------------------------
class Card(ABC):
    def __init__(self, type_="", team=0, mana_cost=0, name="", hidden=False):
        self.type = type_
        self.team = team
        self.mana_cost = mana_cost
        self.name = name
        self.hidden = hidden
        self.deployed = False
        self.dead = False
        self.square = None
        self.exhausted=False
        self.health=0
        self.initial_health=0
        self.defeffects=[]
    
    def get_possible_moves(self, game, player):
        self.player=player
    def die(self, player):
        """Troop dies and returns to largedeck for reuse."""
        
        self.deployed = False
        if self in player.active_cards:
            player.active_cards.remove(self)
        if self.square:
            self.square.troop=None
            self.square.empty=True
        if self in player.deck:
            player.deck.remove(self)
        player.largedeck.appendleft(self)  # Recycle the card
        self.square = None
        self.health=self.initial_health
    def take_damage(self,caster,damage,player):
        self.damage=damage
        for effect in self.defeffects:
            effect.tick(self,caster,self.damage)
        self.health-=self.damage
        print(f"Player{self.team+1}'s {self.name} on Square {self.square.index} took {self.damage} damage! {self.health} remaining. ")
        if self.health<=0:
            self.die(self.player)
            print("It died!")
# -----------------------------
# WizardCard
# -----------------------------
class WizardCard(Card):
    def __init__(self, team=0):
        super().__init__(type_="troop", team=team, mana_cost=6,name="Wizard")
        self.initial_health=8
        self.health = 8
        self.hidden=False
        self.exhausted=False
        self.ability_cost=6

    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq,player))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= self.ability_cost and not self.exhausted and self.deployed:
                for sq in opponent.available_squares:
                    moves.append(FireballMove(self, sq,player))
        return moves
class ThornCard(Card):
    def __init__(self, team=0):
        super().__init__(type_="spell", team=team, mana_cost=4,name="ThornSpell")
        self.initial_health=100
        self.health = 100
        self.hidden=False
        self.exhausted=False
        self.ability_cost=100
    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if not sq.empty:
                    moves.append(ThornMove(self, sq,player))
        return moves

# -----------------------------
# WizardCard
# -----------------------------
class GhostCard(Card):
    def __init__(self, team=0):
        super().__init__(type_="troop", team=team, mana_cost=4, name="Ghost")
        self.health = 4
        self.initial_health=4
        self.hidden=True
        self.exhausted=False
        self.ability_cost=2
        self.damage=1
        
    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq,player))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= self.ability_cost and not self.exhausted and self.deployed:
                for sq in opponent.available_squares:
                    moves.append(HauntMove(self,sq,self.damage,self.ability_cost,player))

        return moves
    def die(self,player):
        self.ability_cost=2
        self.damage=1
        super().die(player)
        
        

class HealCard(Card):
    def __init__(self, team=0):
        super().__init__(type_="spell", team=team, mana_cost=3, name="HealSpell")
        self.health = 1000
        self.hidden=True
        self.exhausted=False
        self.ability_cost=1000
    def get_possible_moves(self, game, player):
        self.player = player
        

        opponent = game.players[1 - player.team]
        if player.mana >= self.mana_cost and not self.exhausted:
             

            return [HealMove(self,player)]
        return []
class ExorcismCard(Card):
    def __init__(self, team=0):
        super().__init__(type_="spell", team=team, mana_cost=4, name="ExorcismSpell")
        self.health = 1000
        self.hidden=True
        self.exhausted=False
        
    def get_possible_moves(self, game, player):
        self.player = player
        

        opponent = game.players[1 - player.team]
        if player.mana >= self.mana_cost and not self.exhausted:
             

            return [ExorcismMove(self,player)]
        return []

In [6]:

class NukeCard(Card):
    def __init__(self, team=0):
        super().__init__(type_="spell", team=team, mana_cost=12, name="NukeSpell")
        self.health = 1000
        self.hidden=True
        self.exhausted=False
        self.ability_cost=1000
    def get_possible_moves(self, game, player):
        self.player = player
        

        opponent = game.players[1 - player.team]
        if player.mana >= self.mana_cost and not self.exhausted:
             

            return [NukeMove(self,player)]
        return []

In [7]:
# -----------------------------
# ExecutionerCard
# -----------------------------
class ExecutionerCard(Card):
    def __init__(self, team=0):
        super().__init__(type_="troop", team=team, mana_cost=4,name="Executioner")
        self.initial_health=12
        self.health = 12
        self.hidden=False
        self.exhausted=False
        self.ability_cost=2
        self.execution_threshold=60

    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq,player))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= self.ability_cost and not self.exhausted and self.deployed:
                for sq in [square for square in opponent.available_squares if not square.empty and not square.troop.hidden]:
                    if sq.troop.health<=.01*self.execution_threshold*sq.troop.initial_health:
                        moves.append(ExecutionerMove(self, sq,player))
        return moves

# -----------------------------
# ExecutionerCard
# -----------------------------

In [8]:
class ManaTowerCard(Card):
    def __init__(self,team=0):
        super().__init__(type_="troop",team=team,mana_cost=4,name="ManaTower")
        self.initial_health=7
        self.health=self.initial_health
        self.hidden=False
        self.exhausted=False
        self.ability_cost=6
        self.effect=None
        self.deployed=False
    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployManaTowerMove(self, sq,player))
        return moves
    def die(self,player):
         
        super().die(player)
        if self.effect in player.passive_effectors:
            player.passive_effectors.remove(self.effect)
        

In [9]:
# -----------------------------
# Square and BuffSquare
# -----------------------------
class Square:
    def __init__(self, index, troop=Card()):
        self.index = index
        self.troop = troop
        self.empty = True

class BuffSquare:
    def __init__(self):
        self.empty = True

In [10]:
# -----------------------------
# Move base class
# -----------------------------
class Move(ABC):
    @abstractmethod
    def execute(self, game):
        pass

# -----------------------------
# DeployTroopMove
# -----------------------------
class DeployTroopMove(Move):
    def __init__(self, card, square,player):
        self.card = card
        self.square = square
        self.mana_cost = card.mana_cost
        self.player=player
        self.type="Deploy"
        self.movename=card.name+str(square.index)
    def execute(self, game):
        player=self.player
        if self.player.mana < self.mana_cost:
            if game.prnt:
                print(f"❌ Not enough mana to deploy {self.card.name}!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        self.square.troop = self.card
        self.square.empty = False
        self.card.deployed = True
        self.card.square = self.square
        if self.card in player.deck:
            player.deck.remove(self.card)
        player.active_cards.append(self.card)
        player.move_played=(self.card.name,self.square.index)
        if game.prnt:
            print(f"🧙 {self.card.name} deployed on square {self.square.index}. Mana: {player.mana}")
class DeployManaTowerMove(Move):
    def __init__(self, card, square,player):
        self.card = card
        self.square = square
        self.card.effect=ManaTowerEffect(player,card)
        self.player=player
        self.mana_cost=self.card.mana_cost
        self.type="Deploy"
        self.movename=card.name+str(square.index)
    def execute(self, game):
        player=self.player
        if self.player.mana < self.mana_cost:
            if game.prnt:
                print(f"❌ Not enough mana to deploy {self.card.name}!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        self.square.troop = self.card
        self.square.empty = False
        self.card.deployed = True
        self.card.square = self.square
        if self.card in player.deck:
            player.deck.remove(self.card)
        player.active_cards.append(self.card)
        if game.prnt:
            print(f"🧙 {self.card.name} deployed on square {self.square.index}. Mana: {player.mana}")
        self.player.passive_effectors.append(self.card.effect)
class ThornMove(Move):
    def __init__(self, card, square,player):
        self.card = card
        self.square = square
        self.card.effect=HiddenThornmailEffect(player,card)
        self.player=player
        self.mana_cost=self.card.mana_cost
        self.type="Spell"
        self.movename=card.name+str(square.index)
    def execute(self,game):
        self.square.troop.defeffects.append(self.card.effect)
        self.player.deck.remove(self.card)
        self.player.mana-=self.mana_cost
        if self.player.game.prnt:
            print(f"Thorn spell casted on player{self.player.team+1}'s {self.square.troop.name} on square {self.square.index}")
        
        

class FireballMove(Move):
    def __init__(self, caster, target_square,player):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = caster.ability_cost
        self.player=player
        self.type="DamageAbility"
        self.name="Fireball"
        self.movename=self.name+str(target_square.index)
    def execute(self, game):
        player=self.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                print("❌ Not enough mana for fireball!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True

        if game.prnt:
            print(f"Player{player.team+1}'s Wizard on square {self.caster.square.index} casts Fireball at enemy square {idx}! Mana left: {player.mana}")

        splash_indices = [i for i in [idx-1, idx, idx+1] if 0 <= i < len(squares)]
        splash_damage = 6 / len(splash_indices)
        overkill = 0

        for i in splash_indices:
            sq = squares[i]
            if not sq.empty and sq.troop and hasattr(sq.troop, "health") and not sq.troop.dead:
                sq.troop.take_damage(self.caster,splash_damage,opponent)
                
                    
                if hasattr(sq.troop, "health"): 
                    if sq.troop.health <= 0:
                        overkill += -sq.troop.health
                        
                        sq.troop.die(opponent)
                        sq.troop = None
                        sq.empty = True
            else:
                overkill += splash_damage  # hitting empty square counts as overkill

        if overkill > 0:
            opponent.base_health -=overkill
            if game.prnt:
                print(f"🏰 {overkill:.1f} overkill damage to opponent base! Remaining: {opponent.base_health:.1f}")

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True
class HauntMove(Move):
    def __init__(self, caster, target_square,damage,mana_cost,player):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = mana_cost
        self.damage=damage
        self.player=player
        self.type="DamageAbility"
        self.name="Haunt"
        self.movename=self.name+str(target_square.index)
    def execute(self, game):
        player=self.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                
                print("❌ Not enough mana for haunt!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True

        if game.prnt:
            print(f"Player{player.team+1}'s Ghost on square {self.caster.square.index} casts Haunt at enemy square {idx}! Mana left: {player.mana}")

        
        if not self.target_square.empty:
            self.target_square.troop.health-=self.damage
            if game.prnt:
                print(f" {self.damage} damage to opponent at square {idx} Remaining: {self.target_square.troop.health:.1f}")
            if self.target_square.troop.health<=0:
                if game.prnt:
                    print(f" {self.target_square.troop.name} at square {idx} has died!")
                self.target_square.troop.die(opponent)
                self.target_square.troop = None
                self.target_square.empty = True
                
        
            
            
        

        
        if game.prnt:
            print(f" {self.damage} damage to opponent base! Remaining: {opponent.base_health:.1f}")    
        self.caster.damage+=1
        self.caster.ability_cost+=2
        opponent.base_health-=self.damage
        

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True
class HealMove(Move):
    def __init__(self,card,player):
        self.card=card
        self.player=player
        self.mana_cost = card.mana_cost
        self.type="NoTargetSpell"
        self.movename=card.name
    def execute(self, game):
        self.player.mana-=self.mana_cost
        if game.prnt:
            print(f"Player {self.player.team+1} casts heal!")
        if self.player.active_cards:
            for card in self.player.active_cards:
                card.health = card.health+2
                if game.prnt and card.square:
                    print(f"Player {self.player.team+1}'s {card.name} on square {card.square.index} is now {card.health} hp!")
        else:
            self.player.base_health+=1
            if game.prnt:
                print(f"Player {self.player.team+1}'s base is healed! Remaining Health:{self.player.base_health} hp!")
            
                  
                
        
            
        self.card.die(self.player)
class ExorcismMove(Move):
    def __init__(self,card,player):
        self.card=card
        self.player=player
        self.mana_cost = card.mana_cost
        self.type="NoTargetSpell"
        self.movename=card.name
    def execute(self, game):
        opponent=game.players[1-self.player.team]
        self.player.mana-=self.mana_cost
        if game.prnt:
            print(f"Player {self.player.team+1} casts Exorcism!")
        if opponent.active_cards:
            for square in opponent.available_squares:
                if not square.empty:
                    if square.troop.name=="Ghost":
                        if game.prnt :
                            print(f"Player {opponent.team+1}'s {square.troop.name} on square {square.index} has been Exorcised!")

                        square.troop.die(opponent)
        elif game.prnt:
            print(f"No ghosts deployed!")
           
            
                  
                
        
            
        self.card.die(self.player)
class NukeMove(Move):
    def __init__(self,card,player):
        self.card=card
        self.player=player
        self.mana_cost = card.mana_cost
        self.type="NoTargetSpell"
        self.movename=card.name
    def execute(self, game):
        opponent=game.players[1-self.player.team]
        self.player.mana-=self.mana_cost
        if game.prnt:
            print(f"Player {self.player.team+1} casts Nuke!")
        if opponent.active_cards:
            for square in opponent.available_squares:
                if not square.empty:
                    
                    if game.prnt :
                        print(f"Player {opponent.team+1}'s {square.troop.name} on square {square.index} got cooked.")
                    square.troop.die(opponent)
        opponent.base_health-=2
        if game.prnt :
            print(f"Player {opponent.team+1}'s base got blasted!{opponent.base_health} hp remaining.")
        self.card.die(self.player)
       
# -----------------------------
# EndTurnMove
# -----------------------------
class EndTurnMove(Move):
    def __init__(self, player):
        self.player = player
        self.type="EndTurn"
        self.movename="EndTurn"

    def execute(self, game):
        # End the current player's turn first
        game.change_turn()
        game.moves_played += 1

        # Refresh the new player's deck and info
        new_player = game.players[game.turn]
        old_player = game.players[1 - game.turn]

        old_player.mana += 2
        old_player.refresh_deck()

        
        new_player.invigorate()
        new_player.refresh_deck()
        
        

        if game.prnt:
            print(f"Player {old_player.team+1} ends turn and gains 2 mana.")
        old_player.employ_effects()
        new_player.record_information(game,old_player)
        if game.prnt:
            print(f"Player {old_player.team+1} has {old_player.mana} mana remaining.")

In [11]:
class ExecutionerMove(Move):
    def __init__(self, caster, target_square,player):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = caster.ability_cost
        self.player=player
        self.execution_threshold=caster.execution_threshold
        self.type="DamageAbility"
        self.name="Execute"
        self.movename=self.name+str(target_square.index)
    def execute(self, game):
        executionThreshhold = self.execution_threshold ######<--PERCENT health threshhold for execution to instantly kill enemy unit-->#####
        sq = self.target_square
        player=self.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                print("❌ Not enough mana/weak targets for execution!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        if game.prnt:
            print(f"Player{player.team+1}'s Executioner on square {self.caster.square.index} casts Execute at enemy square {self.target_square.index}! ")
        if sq.empty or not sq.troop or not hasattr(sq.troop, "health") or sq.troop.dead:
            if game.prnt:
                print("❌ No targets for execution!")
            return
        if self.target_square.troop.health >= self.target_square.troop.initial_health * (executionThreshhold * 0.01) :
            if game.prnt:
                print("Target is not weak enough for Executioner to execute")
            return
        if game.prnt:
            print(f"Executioner executes {self.target_square.troop.name}  !!! OWNED!!!!")
        
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        opponent.base_health-=2
        if game.prnt:
            print(f"Player {opponent.team+1}'s base takes 2 damage! {opponent.base_health} hp remaining!'")
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True
        self.target_square.troop.die(opponent)
        self.target_square.troop = None
        self.target_square.empty = True
        if game.prnt:
            print(f"Mana left: {player.mana}")
       

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RecurrentPolicy(nn.Module):
    def __init__(self, input_dim, hidden_dim, action_dim):
        super().__init__()

        self.fc_in = nn.Linear(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, action_dim)

    def forward(self, obs, hidden, action_mask):
        """
        obs: (batch, seq_len, input_dim)
        hidden: (h, c) or None
        action_mask: (batch, action_dim)
        """

        x = F.relu(self.fc_in(obs))        # (batch, seq_len, hidden)
        x, hidden = self.lstm(x, hidden)  # x: (batch, seq_len, hidden)

        last_output = x[:, -1]            # use last timestep
        logits = self.fc_out(last_output) # (batch, action_dim)

        # mask illegal actions with -inf
        logits = logits.masked_fill(action_mask == 0, float('-inf'))

        probs = F.softmax(logits, dim=-1)
        return probs, hidden

In [13]:
def onehotmoves(moves):
        #function takes in a list of move instances and gives a onehot encoding of each
        movenames=[move.movename for move in moves]#contains the NAMES of moves rather than instances ex:"Wizard1" 
        movelist=encodemoves(movenames)#one-hot encoded list of possible moves

        return movelist
def convertanswer(possible_moves, action_index):
    # lookup name of the global move at this index
    movename = allmoves[action_index]

    # find the move instance in the legal list
    for move in possible_moves:
        if move.movename == movename:
            return move

    # fallback (should never happen)
    return possible_moves[0]
    

# -----------------------------
# DecisionMaker (random for now)
# -----------------------------
class DecisionMaker():
    def __init__(self):
        pass
    def reset(self):
        pass
class DumbDecisionMaker(DecisionMaker):
    def __init__(self,name="Dummy"):
       
        self.name=name
    def choose_move(self,possible_moves,information):
        return np.random.choice(possible_moves)
        
class NeuralDecisionMaker(DecisionMaker):
    def __init__(self, policy=None,state_dim=infolength, hidden_dim=128, action_dim=len(allmoves), name="BigBrain"):
            
        if policy is None:
                
            self.policy = RecurrentPolicy(input_dim=state_dim,
                                         hidden_dim=hidden_dim,
                                         action_dim=action_dim)
        else:
            self.policy=policy
                
        self.name = name
        self.trajectory = []
        self.hidden = None  # LSTM hidden state
    def reset(self):
        self.hidden = None    # call this at episode start

    def choose_move(self, possible_moves, information):
        # information is your flat 87-dim state
        state = torch.tensor(information, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        # shape = (1, 1, 87)

        action_mask = torch.tensor(onehotmoves(possible_moves), dtype=torch.float32).unsqueeze(0)
        # shape = (1, 37)

        # forward LSTM
        probs, self.hidden = self.policy(state, self.hidden, action_mask)

        dist = torch.distributions.Categorical(probs)
        action = dist.sample()

        # save log-prob for REINFORCE
        self.trajectory.append({
            "log_prob": dist.log_prob(action),
        })

        return convertanswer(possible_moves, action.item())
def reinforce_update(policy, optimizer, trajectory, reward,health1,mana1,mana2):
    if len(trajectory) == 0:
        return
    
    
    
    
    reward_tensor = torch.tensor(reward, dtype=torch.float32)
    loss = torch.stack([-step["log_prob"] * reward_tensor for step in trajectory]).sum()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    trajectory.clear()

In [14]:
# -----------------------------
# Example simulation
# -----------------------------
class simulator():
    def __init__(self,player1, player2, length=20, prnt=True, offset=[0,0],show=False):
        self.player1=player1
        self.player2=player2
        self.length=length
        self.prnt=prnt
        self.offset=offset
        self.show=show
    def simulate(self):
      
        game = Game(self.player1, self.player2, self.length, self.prnt, self.offset,self.show)
        reward=game.play()
        
        
        returndict={"reward":reward,
                   "health1":game.player_1.base_health,
                   "mana1":game.player_1.mana,
                   "health2":game.player_2.base_health,
                   "mana2":game.player_2.mana}
        return returndict

In [15]:
import numpy as np
from collections import Counter
defaultlimits={
    WizardCard:2,
    GhostCard:4,
    ManaTowerCard:4,
    HealCard:4,
    ExecutionerCard:3,
    NukeCard:2,
    ExorcismCard:3
}
def make_deck(team=0, size=20, cards=None, prnt=True, limits=defaultlimits):

    l = []

    if not cards:
        cards = [ThornCard,WizardCard, GhostCard, HealCard, ExorcismCard, ExecutionerCard,ManaTowerCard,NukeCard]

    # Default: no limits (infinite)
    for card in cards:
        if card not in limits:
    
            limits[card] = float('inf')

    # Keep count of how many of each card we've added
    counts = {c: 0 for c in cards}

    while len(l) < size:
        # Choose from cards that haven't hit their limit yet
        available = [c for c in cards if counts[c] < limits[c]]
        if not available:
            raise ValueError("No more cards available — all limits reached before deck filled.")

        card_cls = np.random.choice(available)
        l.append(card_cls(team=team))
        counts[card_cls] += 1

    if prnt:
        print_deck(l)
    return l


def print_deck(deck=None):
    if not deck:
        deck=make_deck()

    print(Counter([item.name for item in deck]))


from IPython.display import display, HTML

display(HTML('''
<style>
.output_scroll {
    height: auto !important;
    max-height: none !important;
}
</style>
'''))

In [ ]:
newlimits={
    WizardCard:8,
    GhostCard:2,
    ManaTowerCard:2,
    HealCard:2,
    ExecutionerCard:2,
    NukeCard:2,
    ExorcismCard:2
}
deck1=make_deck(0,limits=newlimits)
deck2=make_deck(1,limits=newlimits)
p1 = GamePlayer(difficulty=1,game=None,deck=deck1, decision_maker=DumbDecisionMaker(), length=len(allmoves))
p2 = GamePlayer(difficulty=1,game=None,deck=deck2, decision_maker=DumbDecisionMaker(), length=len(allmoves))
sim = simulator(p1, p2, length=50, prnt=True,show=True)
simresult = sim.simulate()


Counter({'Wizard': 5, 'ThornSpell': 3, 'Ghost': 2, 'ExorcismSpell': 2, 'HealSpell': 2, 'NukeSpell': 2, 'ManaTower': 2, 'Executioner': 2})
Counter({'ThornSpell': 5, 'Wizard': 5, 'NukeSpell': 2, 'HealSpell': 2, 'Executioner': 2, 'ManaTower': 2, 'ExorcismSpell': 1, 'Ghost': 1})

--- Turn 0, Player 1 ---
🧙 Wizard deployed on square 2. Mana: 24
----------------------------------------------
Current Board:
Player 2 Health:15
Player 2 Mana:30
Player 2 cards in hand:
Ghost Wizard ManaTower ThornSpell ExorcismSpell
Player 2 troops deployed:
____ ____ ____ ____
Player 1 troops deployed:
____ ____ Wizard ____
Player 1 cards in hand:
ThornSpell Wizard ThornSpell Wizard
Player 1 Health:15
Player 1 Mana:24
----------------------------------------------

--- Turn 0, Player 1 ---
Thorn spell casted on player1's Wizard on square 2
----------------------------------------------
Current Board:
Player 2 Health:15
Player 2 Mana:30
Player 2 cards in hand:
Ghost Wizard ManaTower ThornSpell ExorcismSpell
Play

In [ ]:
import time
import copy
resultholder={}

ddm = DumbDecisionMaker()
toplayagainst = [ddm]
totaltime = 0

for newmodel in range(10):
    
    

    # create new policy + DM
    
    
    
    
    dm_new = NeuralDecisionMaker(name=f"Model{newmodel+1}")
    optimizer = torch.optim.Adam(dm_new.policy.parameters(), lr=.002)
    
    
        


     # pick opponent
    
    
    print(f"\n=== TRAINING MODEL #{dm_new.name}")
    start=time.time()
    maxepisodes=800*(1+newmodel)
    for episode in range(maxepisodes):
        dm_opponent = np.random.choice(toplayagainst)
        



        # fresh decks
        deck1 = make_deck(0, size=20, prnt=False)#
        deck2 = make_deck(1, size=20, prnt=False)
        #deck1=[GhostCard(team=0)]*8 +[ExorcismCard(team=0)]*1 + [NukeCard(team=0)]*6
        #deck2=[GhostCard(team=1)]*8 +[ExorcismCard(team=1)]*1 + [NukeCard(team=1)]*6
        # CLONE the opponent DM (important!)
        # build fresh agent using the policy of dm_opponent
        if isinstance(dm_opponent, NeuralDecisionMaker):


            dm2 = NeuralDecisionMaker(policy=copy.deepcopy(dm_opponent.policy), name=dm_opponent.name)
            
            

        else:
            dm2 = DumbDecisionMaker()

        # reset only the NEW one
        dm_new.reset()
        dm2.reset()
        if f"{dm_new.name}" not in resultholder.keys():
            resultholder[f"{dm_new.name}"]={}
        if f"{dm2.name}" not in resultholder[f"{dm_new.name}"].keys():
            resultholder[f"{dm_new.name}"][f"{dm2.name}"]=[]

        # assign players
        if episode % 2 == 0:
            p1 = GamePlayer(difficulty=1,game=None,deck=deck1, decision_maker=dm_new, length=len(allmoves))
            p2 = GamePlayer(difficulty=1,game=None,deck=deck2, decision_maker=dm2, length=len(allmoves))
            sim = simulator(p1, p2, length=50, prnt=False,show=False)
            simresult = sim.simulate()
            reward=simresult["reward"]
            health=simresult["health1"]
            mana1=simresult["mana1"]
            mana2=simresult["mana2"]
        else:
            p1 = GamePlayer(difficulty=1,game=None,deck=deck2, decision_maker=dm2, length=len(allmoves))
            p2 = GamePlayer(difficulty=1,game=None,deck=deck1, decision_maker=dm_new, length=len(allmoves))
            sim = simulator(p1, p2, length=50, prnt=False,show=False)
            simresult = sim.simulate()
            reward=-simresult["reward"]
            health=simresult["health2"]
            mana1=simresult["mana2"]
            mana2=simresult["mana1"]
        # update only new model
        reinforce_update(dm_new.policy, optimizer, dm_new.trajectory, reward,health,mana1,mana2)

        
        if (episode%50==0 and episode>0):
            elapsed = time.time() - start
            totaltime += elapsed
            print(f"Episodes {episode-50} to {episode} took {elapsed:.2f}s, total {totaltime:.2f}s")
            start = time.time()
        if episode==maxepisodes-1:
            elapsed = time.time() - start
            totaltime += elapsed
            print(f"Final training took {elapsed:.2f}s, total {totaltime:.2f}s")
            start = time.time()
            

        resultholder[f"{dm_new.name}"][f"{dm2.name}"].append(reward)
    toplayagainst.append(dm_new)
        

In [ ]:
print("1=Win,-1=Loss,0=Draw")
print(Counter(resultholder["Model3"]["Model2"]))

In [ ]:
import pandas as pd


In [ ]:
winrates={}
for key1 in resultholder.keys():
    
    num_ones=0
    ln=0
    for key2 in resultholder[key1].keys():
        num_ones+=Counter(resultholder[key1][key2])[1]
        ln+=len(resultholder[key1][key2])
    winrates[key1]=num_ones/ln
winrates    

In [ ]:
 resultholder.keys()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import deque



# results is a list of 1=Win, 0=Draw, -1=Loss
for key1 in sorted(resultholder.keys()):
    for key2 in sorted(resultholder[key1].keys()):
        results_array = np.array(resultholder[key1][key2])

        # -----------------------------
        # Option 1: Color-coded scatter plot
        # -----------------------------
        colors = ['green' if r==1 else 'red' if r==-1 else 'blue' for r in results_array]

        # -----------------------------
        # Option 2: Rolling win-rate (smoother)
        # -----------------------------
        window = 50  # rolling window
        win_rate = [np.mean(results_array[max(0,i-window):i+1]==1) for i in range(len(results_array))]

        plt.figure(figsize=(12,4))
        plt.plot(win_rate, color='green')
        plt.plot((0,len(results_array)),(.5,.5))
        plt.xlabel('Episode')
        plt.ylabel('Win Rate (last 50 episodes)')
        plt.title(f'Rolling Win Rate {key1} Vs {key2}')
        plt.ylim(0,1)
        plt.xlim(0,len(results_array))
        plt.show()

In [ ]:
# results is a list of 1=Win, 0=Draw, -1=Loss
results_array = np.array(resultholder["Model2"]["Model1"])

# -----------------------------
# Option 1: Color-coded scatter plot
# -----------------------------
colors = ['green' if r==1 else 'red' if r==-1 else 'blue' for r in results_array]

# -----------------------------
# Option 2: Rolling win-rate (smoother)
# -----------------------------
window = 20  # rolling window
win_rate = [np.mean(results_array[max(0,i-window):i+1]==1) for i in range(len(results_array))]

plt.figure(figsize=(12,4))
plt.plot(win_rate, color='green')
plt.plot((0,len(results_array)),(.5,.5))
plt.xlabel('Episode')
plt.ylabel('Win Rate (last 50 episodes)')
plt.title('Rolling Win Rate Model 2 vs Model 1')
plt.ylim(0,1)
plt.xlim(0,len(results_array))
plt.show()
# results is a list of 1=Win, 0=Draw, -1=Loss
results_array = np.array(resultholder["Model4"]["Model2"])

# -----------------------------
# Option 1: Color-coded scatter plot
# -----------------------------
colors = ['green' if r==1 else 'red' if r==-1 else 'blue' for r in results_array]

# -----------------------------
# Option 2: Rolling win-rate (smoother)
# -----------------------------
window = 20  # rolling window
win_rate = [np.mean(results_array[max(0,i-window):i+1]==1) for i in range(len(results_array))]

plt.figure(figsize=(12,4))
plt.plot(win_rate, color='green')
plt.plot((0,len(results_array)),(.5,.5))
plt.xlabel('Episode')
plt.ylabel('Win Rate (last 50 episodes)')
plt.title('Rolling Win Rate Model 3 vs. Model 2')
plt.ylim(0,1)
plt.xlim(0,len(results_array))
plt.show()

In [ ]:
print("The following is the list of snapshots")
print("Columns contain both players' health, both players' mana, and one-hot encoded lists of cards in hand.")
'''snapshot = {

"My_Mana": [self.mana]*length,
"Their_Mana": [opponent.mana]*length,
"My_Health": [self.base_health]*length,
"Their_Health": [opponent.base_health]*length,
"Their_Card_Types_Seen":encodehand(enemy_list) ,
"Their_Currently_Visible_Cards": encodedeployed(opponent.available_squares),
"Active_Cards":encodedeployed(self.available_squares),
"Deck":encodehand(self.deck),
"Action Taken":encodemove(self.move)
}'''
import torch



In [ ]:
1e-2